In [ ]:
import json
from typing import Tuple
import numpy as np
import cv2
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.body_parts import core_body_parts

<h1> Load sample Dale P3 pre-swap data </h1>

In [ ]:
s3 = S3AccessUtils('/root/data')
rds = RDSAccessUtils()

In [ ]:
query = """
    SELECT *
    FROM 
        prod.biomass_computations
    WHERE
        pen_id = 145 AND
        akpd_score > 0.95 AND
        captured_at BETWEEN '2021-02-01' and '2021-02-05'
"""

df = rds.extract_from_database(query)

<h1> Unrectify and re-rectify key-points </h1>

<h2> Define helper methods </h2>

In [ ]:
def load_params(params):
    print("Loading params...")
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)

    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)

    print("Params loaded.")
    return left_maps, right_maps

IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000

def get_camera_parameters(params: dict) -> Tuple:
    """Return individual camera parameters from JSON stereo parameters contents."""
    
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()
    
    imageSize = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, 
                                                               distCoeffs2, imageSize, R, T, None, None, 
                                                               None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1, cameraMatrix2, distCoeffs2, R2, P2

In [ ]:
stereo_parameters_o_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40020184_R40029797/2020-09-17T08:34:34.152593000Z_L40020184_R40029797_stereo-parameters.json'
stereo_parameters_n_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40020184_R40029797/2021-02-19T13:20:20.603005000Z_L40020184_R40029797_stereo-parameters.json'



In [ ]:
stereo_parameters_o_f, _, _ = s3.download_from_url(stereo_parameters_o_url)
stereo_parameters_n_f, _, _ = s3.download_from_url(stereo_parameters_n_url)

stereo_params_o = json.load(open(stereo_parameters_o_f))
stereo_params_n = json.load(open(stereo_parameters_n_f))
left_maps_o, right_maps_o, cameraMatrix1_o, distCoeffs1_o, R1_o, P1_o, cameraMatrix2_o, distCoeffs2_o, R2_o, P2_o = get_camera_parameters(stereo_params_o)
left_maps_n, right_maps_n, cameraMatrix1_n, distCoeffs1_n, R1_n, P1_n, cameraMatrix2_n, distCoeffs2_n, R2_n, P2_n = get_camera_parameters(stereo_params_n)

ann_u_rs = []
for idx, row in df.iterrows():
    ann = row.annotation
    
    # un-rectify with matlab params, re-rectify with circular params
    ann_u_r = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = cv2.undistortPoints(np.array([[left_maps_o[0][y, x]]]).astype(float), 
                                    cameraMatrix1_n, distCoeffs1_n, R=R1_n, P=P1_n)[0][0]
            elif side == 'rightCrop':
                x_new, y_new = cv2.undistortPoints(np.array([[right_maps_o[0][y, x]]]).astype(float), 
                                    cameraMatrix2_n, distCoeffs2_n, R=R2_n, P=P2_n)[0][0]
                
            x_new, y_new = int(round(x_new)), int(round(y_new))
            ann_u_r[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })
    
    ann_u_rs.append(ann_u_r)

df['annotation_u_r'] = ann_u_rs

In [ ]:
mean_y_diffs = []
mean_x_diffs = []
for idx, row in df.iterrows():
    ann = row.annotation
    ann_left_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    ann_right_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    x_diffs, y_diffs = [], []
    for bp in core_body_parts:
        x_diffs.append(ann_left_dict[bp][0] - ann_right_dict[bp][0])
        y_diffs.append(ann_left_dict[bp][1] - ann_right_dict[bp][1])
        
    
    mean_x_diffs.append(np.mean(x_diffs))
    mean_y_diffs.append(np.mean(y_diffs))
    
df['mean_x_diff'] = mean_x_diffs
df['mean_y_diff'] = mean_y_diffs



In [ ]:
mean_y_diffs = []
mean_x_diffs = []
for idx, row in df.iterrows():
    ann = row.annotation_u_r
    ann_left_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    ann_right_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    x_diffs, y_diffs = [], []
    for bp in core_body_parts:
        x_diffs.append(ann_left_dict[bp][0] - ann_right_dict[bp][0])
        y_diffs.append(ann_left_dict[bp][1] - ann_right_dict[bp][1])
        
    
    mean_x_diffs.append(np.mean(x_diffs))
    mean_y_diffs.append(np.mean(y_diffs))
    
df['mean_x_diff_u_r'] = mean_x_diffs
df['mean_y_diff_u_r'] = mean_y_diffs



In [ ]:
df.mean_y_diff.mean()

In [ ]:
df.mean_y_diff_u_r.mean()

In [ ]:
df.mean_x_diff.mean()

In [ ]:
(df.mean_x_diff_u_r - df.mean_x_diff)